In [ ]:
import os
import shutil
from mutagen.easyid3 import EasyID3
from mutagen.mp3 import MP3
from mutagen.id3 import APIC, ID3
import mimetypes

In [ ]:
def print_mp3_metadata(mp3_file_path):
    # Load the MP3 file
    audio = MP3(mp3_file_path, ID3=EasyID3)

    # Print all available metadata properties
    print("Available metadata properties:")
    for key in audio.keys():
        print(f"{key}: {audio[key]}")

# Example usage:
mp3_file_path = "D:\\utilities\\mp3-tag-editor\\updated_Akon\\Beautifull.mp3"
print_mp3_metadata(mp3_file_path)

In [ ]:
def update_mp3_metadata(album_path, output_path, metadata={}):
    """
    Update metadata of MP3 files.

    Args:
        album_path (str): Path to the directory containing MP3 files.
        output_path (str): Path to the directory where updated MP3 files will be saved.
        metadata (dict): Metadata to be updated for the MP3 files.
            Example:
            {
                'album': 'Album Name',
                'artist': 'Artist Name',
                'title': 'Title of Song',
                'tracknumber': '1',
                'genre': 'Genre',
                'date': '2023',
                'albumArt': 'Path to album art image file',
            }
    """
    # If output path is not provided, create a default output directory
    if output_path == '':
        output_path = os.path.join(os.getcwd(), f"updated_{os.path.basename(album_path)}")
        os.makedirs(output_path, exist_ok=True)

    # Get a list of all MP3 files in the album path
    mp3_files = [file for file in os.listdir(album_path) if file.endswith(".mp3")]

    # Counter to track the order of files
    track_number = 1

    for mp3_file in mp3_files:
        # Read the MP3 file
        file_path = os.path.join(album_path, mp3_file)

        # Create a new MP3 file with the same audio data
        new_file_path = os.path.join(output_path, mp3_file)
        shutil.copy2(file_path, new_file_path)
        new_audio = MP3(new_file_path, ID3=EasyID3)

        # Update MP3 properties if available
        if metadata:
            for key, value in metadata.items():
                if key != 'albumArt' and value:
                    new_audio[key] = value

        # Update track number
        new_audio['tracknumber'] = str(track_number)

        # Save the changes
        new_audio.save()

        # Update album art if available
        if 'albumArt' in metadata:
            album_art_path = metadata['albumArt']
            if album_art_path:
                mime_type, _ = mimetypes.guess_type(album_art_path)
                new_audio = MP3(new_file_path, ID3=ID3)
                new_audio.tags.add(APIC(
                    encoding=0,
                    mime=mime_type,
                    type=3,
                    desc=u'Cover',
                    data=open(album_art_path,'rb').read()
                ))
                new_audio.save()

        # Print a message indicating the properties are updated
        print(f"Updated properties for {mp3_file} and saved to {new_file_path}")
        track_number += 1

In [ ]:

album_path = 'D:\\utilities\\mp3-tag-editor\\testalbums\\Akon'
output_path = ''

# Example usage:
metadata = {
    'album': '',
    # 'encodedby': 'test',
    'title': 'Beautiful',
    'version': 'test',
    'artist': 'test',
    'albumartist': 'test',
    'genre': 'test',
    'date': '2023',
    'website': 'test',
    'albumArt': r"D:\utilities\mp3-tag-editor\download.jpg"
}

update_mp3_metadata(album_path, output_path, metadata)